In [ ]:
%matplotlib inline
import os
import tensorflow as tf
import numpy as np
import pandas as pd
from scipy.fftpack import dct
import soundfile as sf
import matplotlib.pyplot as plt
from python_speech_features.base import mfcc
import scipy.io.wavfile
from model_new import Seq2SeqModel, train_on_copy_task

In [ ]:
root = '/mnt/disks/sec/'

In [ ]:
class FileOpen(tf.gfile.Open):
    def seek(self, position, whence = 0):
        if (whence == 0):
            tf.gfile.Open.seek(self, position)
        elif (whence == 1):
            tf.gfile.Open.seek(self, self.tell() + position)
        else:
            raise FileError

In [ ]:
mfccs = {}
i = 0
for file in tf.gfile.Open(root + 'wsj0/transcripts/wsj0/wsj0.ctl').readlines():
    #print(file)
    sample_rate, signal = scipy.io.wavfile.read(FileOpen(root + file.rstrip('\n'), 'rb'))
    #sample_rate, signal = scipy.io.wavfile.read(root + file.rstrip('\n'))
    mfccs[file] = mfcc(signal, sample_rate)
    if (i % 1000 == 0):
        print(i)
    i += 1
    #plt.imshow(mfccs[file].T)

In [ ]:
np.savez(mfccs)

In [ ]:
tf.reset_default_graph()
tf.set_random_seed(1)

with tf.Session() as session:

    # with bidirectional encoder, decoder state size should be
    # 2x encoder state size
    model = Seq2SeqModel(encoder_cell=LSTMCell(10),
                         decoder_cell=LSTMCell(20), 
                         vocab_size=10,
                         embedding_size=10,
                         attention=True,
                         bidirectional=True,
                         debug=False)

In [ ]:
num_epochs = 2
for epoch in range(num_epochs):
    for line in tf.gfile.Open(root + 'wsj0/transcripts/wsj0/wsj0.faligned.trans').readlines():
        file = line.split()[-1][1:-1]
        data = line.split()[:-1]
        break